In [1]:
import tqdm
from rtc_transit_equity.datasets import generate
data = generate()
print(data.keys())

Finished all dataset gathering and preprocessing in 3.6197919845581055s
dict_keys(['route_df', 'ridership_df', 'tract_population_df', 'county_population_df', 'bus_stop_income', 'result'])


In [2]:
result = data['result']
ridership_df = data['ridership_df']

# Strategic Q1
p = population census count of some bus stop

m = median household income of some bus stop

We defined the median household income of a route to be:
$$\frac{\sum p * m}{\sum p}$$

Where we are essentially normalizing the median household income of a route based on the stops along that route

In [3]:
route_ids = list(set(result.route_id))

# TODO FIGURE OUT WHY WE HAVE DUPLICATE AGENCIES
# Normalize route's median household income by bus stops along the route
averages = {}
for route_id in tqdm(route_ids):
    route_stops = result.loc[result.route_id == route_id]
    route_avg = (route_stops.population * route_stops.median_household_income).sum() / route_stops.population.sum()
    
    agencies = list(set(route_stops.Agency))
    averages[route_id] = {
        "median_household_income": float(route_avg), 
        "RTA": list(set(route_stops.Agency)),
        "route_population": float(route_stops.drop_duplicates('census_tract').population.sum()),
        "short_name": list(set(route_stops.route_short_name))
    }


NameError: name 'tqdm' is not defined

In [ ]:
vals = list(averages.items())
poorest_routes = sorted(vals, key=lambda elem: elem[1]["median_household_income"])
print(f"Poorest routes:\n\n{json.dumps(poorest_routes[:10], indent=2)}")

# Strategic Q2

C: Total Operating Cost

T: Total number of unlinked trips

F: Total fares in a fiscal year

Average Cost = $$\frac{C}{T}$$

Free Estimated Average Cost Per Trip = $$\frac{C + F}{T * 1.3}$$

We use the 1.3 constant due to the simpson curtson rule

In [ ]:
RIDERSHIP_INCREASE_CONSTANT = 1.3

ridership_df['Free Estimated Average Cost Per Trip'] = \
    (ridership_df['Operating Expenses FY'] + ridership_df['Fares FY']) / \
    (ridership_df['Unlinked Passenger Trips FY'] * RIDERSHIP_INCREASE_CONSTANT)

ridership_df['Free percent average cost change'] = \
    (ridership_df['Free Estimated Average Cost Per Trip'] - ridership_df['Average Cost per Trip FY']) / \
    ridership_df['Average Cost per Trip FY']

In [ ]:
display(ridership_df[[
    'Free Estimated Average Cost Per Trip',
    'Average Cost per Trip FY',
    'Free percent average cost change',
    'Agency'
]].sort_values('Free percent average cost change').dropna())

Below I will prove that assuming we have a 30% increase of ridership when we remove 100% of fares via the simpson curtson rule, average cost per trip will always decrease when the amount of revenue you generate is less than 3 tenths of the total operational cost:

I will define the below function to be the percent change of average cost per trip when you remove 100% of fares. Let's define the following variables:

x: Total revenue from fares

C: Total cost of operation

T: Total number of trips

$$g(x) = \frac{(\frac{c + x}{t*1.3}) - \frac{c}{t}}{\frac{c}{t}}$$

Notice this function will be strictly increasing when C and T are positive numbers.

Now to solve for when y=0:

$$0 = \frac{(\frac{c + x}{t*1.3}) - \frac{c}{t}}{\frac{c}{t}}$$

$$0 = \frac{c + x}{t*1.3} - \frac{c}{t}$$

$$\frac{c}{t} = \frac{c + x}{t*1.3}$$

$$c * 1.3 = c + x$$

$$c * 1.3 - c = x$$

$$.3 * c = x$$